In [1]:
import pandas as pd
from sklearn.linear_model import LassoCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression
import numpy as np

In [2]:
# Quality of Government 
# Codebook: https://www.qogdata.pol.gu.se/data/codebook_std_jan23.pdf
# Covid Data - jht_ccd: Number of COVID-19 deaths reported
# Cross-section min. year: 2020; Cross-section max. year: 2020; N. of countries: 193
qog = pd.read_csv('https://www.qogdata.pol.gu.se/data/qog_std_cs_jan23.csv') 

#COVID-19 Data Repository by the Center for Systems Science and Engineering (CSSE) at Johns Hopkins University - deaths
jhu_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jhu_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')

# Oxford Covid-19 Government Response Tracker
owid = pd.read_csv('https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/owid-covid-data.csv')
policy_index = pd.read_csv('https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/OxCGRT_nat_latest.csv')

### Variables 
#### Dependent Variables in LASSO regression

**skeleton** 
$$
\text{Covid Death Rate} = \frac{\text{Total number of Covid deaths}}{\text{Population}}
$$


$$
\text{Covid Infection Rate} = \frac{\text{Total number of Covid cases}}{\text{Population}}
$$

In regression code, the independent variables are defined as: 
$$
\text{target_death} = \frac{\text{jht_ccd}}{\text{wdi_pop}}
$$

$$
\text{target_case} = \frac{\text{jht_ccc}}{\text{wdi_pop}}
$$




**4.8 COVID-19 Data Repository**
Dataset by: Center for Systems Science and Engineering
If you use any of these variables, make sure to cite the original source and QoG Data. Our
suggested citation for this dataset is:

Ensheng, D., Du, H., & Gardner, L. (2020). An interactive web-based dashboard to track covid19 in real time. The Lancet, 20 (5), 533–534. https://doi.org/10.1016/S1473-3099(20)30120-1
Dataset found at: https://github.com/CSSEGISandData/COVID-19
Last update by original source: 2022-12-12
Date of download: 2022-12-12


The data repository for the 2019 Novel Coronavirus Visual Dashboard operated by the Johns Hopkins University Center for Systems Science and Engineering (JHU CSSE). Also, Supported by ESRI
Living Atlas Team and the Johns Hopkins University Applied Physics Lab (JHU APL).

**4.8.1 Number of COVID-19 cases reported**

QoG Code: jht_ccc

This is the number of reported cases of COVID-19 during the year.
Type of variable: Discrete
**Available in Cross-section**

Cross-section min. year: 2020; Cross-section max. year: 2020

N. of countries: 193


**4.8.2 Number of COVID-19 deaths reported**

QoG Code: jht_ccd

This is the number of reported deaths due to COVID-19 during the year.
Type of variable: Discrete

**Available in Cross-section**
Cross-section min. year: 2020; Cross-section max. year: 2020

N. of countries: 193




**4.116.210 Population, total** (Dataset version: 2023 Jan) 

QoG Code: wdi_pop

Total population is based on the de facto definition of population, which counts all residents regardless of legal status or citizenship. The values shown are midyear estimates.


Type of variable: Discrete

**Available in Cross-section**

Cross-section min. year: 2019; Cross-section max. year: 2019

N. of countries: 193

**Available in Time-series**

Time-series min. year: 1960; Time-series max. year: 2021
Total N. of countries covered: 200

In [3]:
qog.shape

(194, 1685)

In [4]:
# Create new columns as per the provided formulas
qog['target_death'] = qog['jht_ccd'] / qog['wdi_pop']
qog['target_case'] = qog['jht_ccc'] / qog['wdi_pop']

# Display the first few rows to confirm the creation of new columns
qog[['jht_ccd', 'jht_ccc', 'wdi_pop', 'target_death', 'target_case']]

,jht_ccd,jht_ccc,wdi_pop,target_death,target_case
0,2189.0,52330.0,37769500.0,0.000058,0.001386
1,1181.0,58316.0,2854191.0,0.000414,0.020432
2,2756.0,99610.0,42705368.0,0.000065,0.002332
3,84.0,8049.0,76343.0,0.001100,0.105432
4,405.0,17553.0,32353588.0,0.000013,0.000543
...,...,...,...,...,...
189,614.0,77060.0,33580352.0,0.000018,0.002295
190,1028.0,113558.0,28971684.0,0.000035,0.003920
191,0.0,2.0,211905.0,0.000000,0.000009
192,610.0,2099.0,31546692.0,0.000019,0.000067


Since the jht_ccc and jht_ccd only contains 
1) number of reported COVID-19 cases during the year 
2) number of reported deaths due to COVID-19 during the year 
Create a new iv and dv with the latest data from JHU

## LASSO Regression with Simple Imputer 

In [5]:
# Preparing the data for LASSO regression
# Simple Imputer 
# Selecting predictors and targets
X_qog = qog.drop(columns=['target_death', 'target_case', 'jht_ccd', 'jht_ccc'])
y_death_qog = qog['target_death']
y_case_qog = qog['target_case']

# Removing non-numeric columns from the dataset
X_numeric_qog = X_qog.select_dtypes(include=[np.number])

# Imputing missing values in the numeric-only dataset
imputer_qog = SimpleImputer(strategy='mean')
X_imputed_qog = imputer_qog.fit_transform(X_numeric_qog)

# Standardizing the predictor variables
scaler_qog = StandardScaler()
X_scaled_qog = scaler_qog.fit_transform(X_imputed_qog)

# LASSO Regression for target_death
X_train_death_qog, X_test_death_qog, y_train_death_qog, y_test_death_qog = train_test_split(X_scaled_qog, y_death_qog, test_size=0.3, random_state=0)
lasso_death_qog = LassoCV(cv=5, random_state=0).fit(X_train_death_qog, y_train_death_qog)

# LASSO Regression for target_cases
X_train_case_qog, X_test_case_qog, y_train_case_qog, y_test_case_qog = train_test_split(X_scaled_qog, y_case_qog, test_size=0.3, random_state=0)
lasso_case_qog = LassoCV(cv=5, random_state=0).fit(X_train_case_qog, y_train_case_qog)

# Getting the coefficients
coefficients_death_qog = lasso_death_qog.coef_
coefficients_case_qog = lasso_case_qog.coef_

/Users/jinwens/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.401e-09, tolerance: 1.805e-09
  model = cd_fast.enet_coordinate_descent(


In [6]:
non_zero_vars_death = X_qog.columns[np.where(coefficients_death_qog != 0)]
non_zero_vars_case = X_qog.columns[np.where(coefficients_case_qog != 0)]

len(non_zero_vars_death), len(non_zero_vars_case)

(22, 36)

In [7]:
non_zero_vars_death

Index(['ccode_qog', 'cbie_cbiconstitution', 'cri_nonopen', 'cses_pc',
       'epi_cch', 'eu_imm65f', 'gdg_resp', 'gendip_nfr', 'gpi_ss', 'h_lflo',
       'h_polcon5', 'ideaesd_lsvm', 'ideavt_eucv', 'ihme_hle_0104m',
       'lis_medeqi', 'nunn_desert', 'ti_cpi_min', 'wdi_wip', 'wgov_tot',
       'wvs_confch', 'wvs_confcs', 'wwbi_spupempp'],
      dtype='object')

In [8]:
non_zero_vars_case

Index(['cpds_vcon', 'dev_altv1', 'dev_tv1', 'dr_sg', 'epi_fsh', 'eu_heaalcnv',
       'eu_heanursnr', 'eu_sctcltclmf', 'eu_trfrldnld', 'gendip_mfr',
       'gol_enpp', 'gol_enpres', 'h_f', 'h_lfup', 'ihme_hle_0104f',
       'ihme_hle_0104m', 'ipu_u_w', 'lis_pr8020', 'nunn_desert',
       'oecd_valaddac_t1d', 'une_surg5pem', 'wbgi_pve', 'wdi_birthreg',
       'wdi_expeduge', 'wdi_expmil', 'wdi_fertility', 'wdi_firftopm',
       'wdi_gersm', 'wdi_qpubadm', 'wdi_spr', 'wgov_minmil', 'wgov_tot',
       'who_let', 'wvs_confjs', 'wvs_demimp', 'wwbi_spupempp'],
      dtype='object')

In [9]:
# Extracting the coefficients for these variables
coefficients = {var: lasso_death_qog.coef_[X_qog.columns.get_loc(var)] for var in non_zero_vars_death}

# Sorting variables by the absolute value of their coefficients
sorted_variables = sorted(coefficients.items(), key=lambda x: abs(x[1]), reverse=True)

sorted_variables = pd.DataFrame(sorted_variables, columns=['Feature', 'Importance'])
sorted_variables

,Feature,Importance
0,gdg_resp,0.000083
1,wwbi_spupempp,0.000061
2,ti_cpi_min,0.000042
3,lis_medeqi,0.000039
4,wvs_confch,-0.000037
5,cri_nonopen,-0.000037
6,ihme_hle_0104m,0.000035
7,wvs_confcs,-0.000024
8,gpi_ss,-0.000017
9,wgov_tot,0.000016


In [10]:
# The most relevant variable will be the first in this sorted list (for target death)
most_relevant_variable = sorted_variables.iloc[0]['Feature']
print("Most relevant variable to target_deaths:", most_relevant_variable)

Most relevant variable to target_deaths: gdg_resp


In [11]:
# Extracting the coefficients for these variables
coefficients = {var: lasso_case_qog.coef_[X_qog.columns.get_loc(var)] for var in non_zero_vars_case}

# Sorting variables by the absolute value of their coefficients
sorted_variables_case = sorted(coefficients.items(), key=lambda x: abs(x[1]), reverse=True)
sorted_variables_case = pd.DataFrame(sorted_variables_case, columns=['Feature', 'Importance'])
sorted_variables_case

,Feature,Importance
0,wwbi_spupempp,0.002599
1,h_lfup,-0.002146
2,dev_tv1,0.002116
3,cpds_vcon,0.001880
4,eu_heaalcnv,0.001658
5,wdi_qpubadm,0.001630
6,nunn_desert,-0.001438
7,wdi_spr,0.001420
8,gendip_mfr,0.001395
9,ihme_hle_0104m,0.001354


In [12]:
# The most relevant variable will be the first in this sorted list (for target death)
most_relevant_variable_case = sorted_variables_case.iloc[0]['Feature']
print("Most relevant variable to target_cases:", most_relevant_variable_case)

Most relevant variable to target_cases: wwbi_spupempp


In [13]:
# Finding the common variables with non-zero coefficients in both models
common_non_zero_vars = set(non_zero_vars_death).intersection(set(non_zero_vars_case))
print("The common variables with non-zero coefficients in both models:", common_non_zero_vars)

The common variables with non-zero coefficients in both models: {'wwbi_spupempp', 'ihme_hle_0104m', 'nunn_desert', 'wgov_tot'}


## Random Forest Regression with Simple Imputer 

In [14]:
# Removing rows with NaN values in 'target_death'
qog_cleaned = qog.dropna(subset=['target_death'])

X = qog_cleaned.select_dtypes(include=[np.number]).drop(['target_death', 'target_case', 'jht_ccd', 'jht_ccc'], axis=1)
y_death = qog_cleaned['target_death']

# Re-imputing and scaling the features
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)

# Splitting the data
X_train_death, X_test_death, y_train_death, y_test_death = train_test_split(X_scaled, y_death, test_size=0.3, random_state=42)

# Train Random Forest model for 'target_death'
rf_death = RandomForestRegressor(n_estimators=100, random_state=42)
rf_death.fit(X_train_death, y_train_death)

# Extracting and sorting feature importances_death
feature_importances_death = rf_death.feature_importances_
feature_names = X.columns
features_death = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importances_death})
features_death_sorted = features_death.sort_values(by='Importance', ascending=False)

features_death_sorted.head(10)


,Feature,Importance
2,ccodecow,0.074567
1582,wvs_confgov,0.039556
1662,yri_agi40,0.035435
1674,yri_mp40,0.025408
1672,yri_mp30,0.023609
1585,wvs_conflu,0.020291
816,lis_medeqi,0.019006
1670,yri_meanage,0.018364
1584,wvs_confjs,0.016999
1579,wvs_confcs,0.016625


In [15]:
# For target_case
# Assuming 'target_case' is another target variable you want to predict
y_case = qog_cleaned['target_case']

# Splitting the data for 'target_case'
X_train_case, X_test_case, y_train_case, y_test_case = train_test_split(X_scaled, y_case, test_size=0.3, random_state=42)

# Training Random Forest model for 'target_case'
rf_case = RandomForestRegressor(n_estimators=100, random_state=42)
rf_case.fit(X_train_case, y_train_case)

# Extracting and sorting feature importances
feature_importances_case = rf_case.feature_importances_
feature_names = X.columns
features_case = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importances_case})
features_case_sorted = features_case.sort_values(by='Importance', ascending=False)

features_case_sorted.head(10)

,Feature,Importance
1519,who_dwtot,0.081699
1274,wdi_gdpagr,0.075960
1210,wdi_birthreg,0.047573
253,dr_eg,0.045253
254,dr_ig,0.032817
1358,wdi_lifexpf,0.029519
1674,yri_mp40,0.019853
775,ihme_hle_0104t,0.015441
1376,wdi_mortu5f,0.011573
317,eu_demgrownnat,0.010085


In [22]:
missing_percentage = qog.isnull().mean() * 100

# Identify columns where more than 30% of the data is missing
columns_to_drop = missing_percentage[missing_percentage > 30].index

# Drop these columns from the DataFrame
qog_test = qog.drop(columns=columns_to_drop)
qog_test = qog_test.select_dtypes(include=[np.number])

In [23]:
qog_test

,ccode,ccode_qog,ccodecow,al_ethnic2000,al_language2000,al_religion2000,atop_ally,atop_consult,atop_defensive,atop_neutrality,...,wjp_ord_secur,wjp_overall,wjp_pol_mil,wjp_ppl_civ_jus,wjp_regul_enforc,wjp_trans_pow,wui_wtui,wui_wui,target_death,target_case
0,4,4,700.0,0.769345,0.614146,0.271684,1.0,1.0,0.0,0.0,...,0.295796,0.347640,0.386950,0.468051,0.350218,0.336787,0.0,0.000000,0.000058,0.001386
1,8,8,339.0,0.220426,0.039925,0.471852,1.0,1.0,1.0,1.0,...,0.790236,0.506076,0.530434,0.515302,0.436254,0.521543,0.0,0.157854,0.000414,0.020432
2,12,12,615.0,0.339400,0.442662,0.009128,1.0,1.0,1.0,0.0,...,0.722973,0.505867,0.472320,0.599516,0.518435,0.491529,0.0,0.096900,0.000065,0.002332
3,20,20,232.0,0.713946,0.684785,0.232569,1.0,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.001100,0.105432
4,24,24,540.0,0.786720,0.787019,0.627644,1.0,1.0,1.0,0.0,...,0.574618,0.412630,0.459923,0.531473,0.416932,0.397987,0.0,0.171155,0.000013,0.000543
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189,860,860,704.0,0.412514,0.411987,0.213264,1.0,1.0,1.0,1.0,...,0.906201,0.463741,0.390256,0.455407,0.437169,0.512761,0.0,0.398779,0.000018,0.002295
190,862,862,101.0,0.496600,0.068580,0.135030,1.0,1.0,1.0,0.0,...,0.478246,0.276299,0.373358,0.444220,0.200404,0.185288,0.0,0.351172,0.000035,0.003920
191,882,882,990.0,0.137608,0.011111,0.787117,1.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000009
192,887,887,679.0,NaN,0.007982,0.002286,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.093860,0.000019,0.000067


In [ ]:
# LASSO with IterativeImputer

# Selecting predictors and targets from qog_test
X_qog_test = qog_test.drop(columns=['target_death', 'target_case', 'jht_ccd', 'jht_ccc'])
y_death_qog_test = qog_test['target_death']
y_case_qog_test = qog_test['target_case']

# Removing non-numeric columns from the dataset
X_numeric_qog_test = X_qog_test.select_dtypes(include=[np.number])

# Imputing missing values in the numeric-only dataset using Iterative Imputer
imputer_qog = IterativeImputer(estimator=LinearRegression(), max_iter=10, random_state=0)
X_imputed_qog = imputer_qog.fit_transform(X_numeric_qog)

In [ ]:
# Standardizing the predictor variables
scaler_qog = StandardScaler()
X_scaled_qog = scaler_qog.fit_transform(X_imputed_qog)

# LASSO Regression for target_death
X_train_death_qog, X_test_death_qog, y_train_death_qog, y_test_death_qog = train_test_split(X_scaled_qog, y_death_qog, test_size=0.3, random_state=0)
lasso_death_qog = LassoCV(cv=5, random_state=0).fit(X_train_death_qog, y_train_death_qog)

# LASSO Regression for target_cases
X_train_case_qog, X_test_case_qog, y_train_case_qog, y_test_case_qog = train_test_split(X_scaled_qog, y_case_qog, test_size=0.3, random_state=0)
lasso_case_qog = LassoCV(cv=5, random_state=0).fit(X_train_case_qog, y_train_case_qog)

# Getting the coefficients
coefficients_death_qog = lasso_death_qog.coef_
coefficients_case_qog = lasso_case_qog.coef_

In [ ]:
non_zero_death_indices = coefficients_death_qog != 0
non_zero_case_indices = coefficients_case_qog != 0

# Creating data frames with non-zero coefficients
# Assuming X_numeric_qog has column names that correspond to features
df_non_zero_death = pd.DataFrame({
    'Feature': X_numeric_qog.columns[non_zero_death_indices],
    'Coefficient': coefficients_death_qog[non_zero_death_indices]
})

df_non_zero_case = pd.DataFrame({
    'Feature': X_numeric_qog.columns[non_zero_case_indices],
    'Coefficient': coefficients_case_qog[non_zero_case_indices]
})